In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import sys
import random

from tqdm import tqdm

import string

import os

import json

In [3]:
!pip install -q transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 60.4 MB/s 
     |████████████████████████████████| 3.3 MB 45.1 MB/s 
     |████████████████████████████████| 596 kB 66.8 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 


In [4]:
from transformers import AutoTokenizer, BertConfig, TFBertModel

In [5]:
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy as sce
from tensorflow.keras.callbacks import Callback

In [6]:
train_ins = '/content/drive/MyDrive/data/fine_data/train_instance_128_clean_975.csv'
val_ins = '/content/drive/MyDrive/data/fine_data/val_instance_128_clean.csv'

train_squad = '/content/drive/MyDrive/data/fine_data/train_instance_squad.csv'

val_id = '/content/drive/MyDrive/data/fine_data/val_id.csv'
train_id = '/content/drive/MyDrive/data/fine_data/train_id.csv'

val_cand = '/content/drive/MyDrive/data/fine_data/val_mapping.csv'
gt = '/content/drive/MyDrive/data/fine_data/validation_long.csv'
gt_short = '/content/drive/MyDrive/data/fine_data/validation_short.csv'

## **Functions to tokenize data**

In [7]:
model_name = '/content/drive/MyDrive/data/bert_base_uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)

tags = [ '``', '\'\'', '--']

# for i in range(0, 51, 1):
#     tags.append(f'[part={i}]')

print(tags)

special_tokens_dict = {'additional_special_tokens': tags}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(num_added_toks)
print(len(tokenizer))

# encoder.resize_token_embeddings(len(tokenizer))

['``', "''", '--']
3
30525


In [8]:
AnswerType = {
    'NO_ANSWER': 0,
    'YES': 1,
    'NO': 2,
    'SHORT' : 3,
    'LONG' : 4
}

def preprocess_data(data, tokenizer, debug=False): 
    progress = tqdm(data, total=len(data))
    x1 = []
    x2 = []
    x3 = []
    y = []
    for sam in progress:
        # part_id = sam['part_id']
        # part_tokens = f'[part={part_id}]'

        # context = part_tokens + " " + sam['context']
        tokenized_sam = tokenizer.encode_plus(sam['question'], sam['context'], 
                                              padding='max_length',
                                              truncation=True,
                                              max_length=512,
                                              add_special_tokens=True)
        
        x1.append(tf.cast(tokenized_sam['input_ids'], tf.int32))
        x2.append(tf.cast(tokenized_sam['token_type_ids'], tf.int32))
        x3.append(tf.cast(tokenized_sam['attention_mask'], tf.int32))

        y.append([sam['start'], sam['stop'], AnswerType[sam['target']]])

    x1 = tf.convert_to_tensor(x1)
    x2 = tf.convert_to_tensor(x2)
    x3 = tf.convert_to_tensor(x3)

    y = tf.convert_to_tensor(y)
    return x1, x2, x3, y


In [ ]:
train_ins_df = pd.read_csv(train_ins)
# train_ins_df = pd.read_csv(train_squad)
tran_ins_list = train_ins_df.to_dict('records')
print(len(tran_ins_list))

577571


In [ ]:
val_id_df = pd.read_csv(val_id)
val_id_list = val_id_df['example_id'].tolist()

val_cand_df = pd.read_pickle(val_cand)
val_cand_list = val_cand_df.to_dict('records') 

gt_df = pd.read_csv(gt)
gt_list = gt_df.to_dict('records')

In [ ]:
val_ins_df = pd.read_csv(val_ins)
val_ins_list = val_ins_df.to_dict('records')
print(len(val_ins_list))

169721


In [9]:
def get_strategy():
    try:
        tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu_cluster_resolver.cluster_spec().as_dict()['worker'])
        tf.config.experimental_connect_to_cluster(tpu_cluster_resolver)
        tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)
        strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)
    except ValueError as e:
        print(e)
        print('No TPU detected')
        tpu = None
        strategy = tf.distribute.get_strategy()
    return strategy

In [10]:
def create_model(tokenizer, model_name, debug=False):
    config = BertConfig()
    if debug:
        print(config)
    # encoder = TFBertModel(config)
    encoder = TFBertModel.from_pretrained(model_name)
    encoder.resize_token_embeddings(len(tokenizer))

    NUM_TARGET = 5
    class MyQAModel(tf.keras.Model):
        def __init__(self, *inputs, **kwargs):
            super().__init__(*inputs, **kwargs)            
            self.bert = encoder

            # self.dropout_start = tf.keras.layers.Dropout(0.1)
            # self.dropout_stop = tf.keras.layers.Dropout(0.1)
            # self.dropout_target = tf.keras.layers.Dropout(0.1)

            self.start_logits = tf.keras.layers.Dense(1)
            self.stop_logits = tf.keras.layers.Dense(1)
            
            self.target = tf.keras.layers.Dense(NUM_TARGET)

        def call(self, inputs, **kwargs):
            bert_res=self.bert(inputs[0], 
                               token_type_ids=inputs[1], 
                               attention_mask=inputs[2]
                               )
            
            # dropout_res1 = self.dropout_start(bert_res[0])
            dropout_res1 = bert_res[0]

            start_logits = tf.squeeze(self.start_logits(dropout_res1), -1)

            # dropout_res2 = self.dropout_stop(bert_res[0])
            dropout_res2 = bert_res[0]

            stop_logits = tf.squeeze(self.stop_logits(dropout_res2), -1)

            # dropout_res3 = self.dropout_target(bert_res[1])
            dropout_res3 = bert_res[1]
            
            targets = self.target(dropout_res3)
            
            paddings = tf.constant([[0, 0,], [0, 512-NUM_TARGET]])
            targets = tf.pad(targets, paddings)
            
            res = tf.stack([start_logits, stop_logits, targets], axis=1)
            return res
        
    model = MyQAModel()
    return model 

In [ ]:
def getFineInsRes(raw_res, ins_list, debug=False):
    list_ins_res = []
    progress = tqdm(ins_list, total=len(ins_list))
    for rowid, row in enumerate(progress):
        example_id = row['example_id']
        part_id = row['part_id']

        res_start = tf.nn.softmax(raw_res[rowid][0]).numpy()
        res_stop = tf.nn.softmax(raw_res[rowid][1]).numpy()
        res_target = tf.nn.softmax(raw_res[rowid][2]).numpy()
        
        start = np.argmax(res_start)
        stop = np.argmax(res_stop)
        target = np.argmax(res_target)

        start_score = res_start[start]
        stop_score = res_stop[stop]

        start_CLS = res_start[0]
        stop_CLS = res_stop[0]
        
        ins_res = {}
        ins_res['example_id'] = example_id
        ins_res['part_id'] = part_id

        ins_res['start'] = start 
        ins_res['stop'] = stop
        ins_res['target'] = target

        ins_res['start_score'] = start_score
        ins_res['stop_score'] = stop_score
        
        ins_res['start_CLS'] = start_CLS
        ins_res['stop_CLS'] = stop_CLS
        if debug:
            if rowid == 101:
                print(row)
                print(ins_res)
        list_ins_res.append(ins_res)
    list_ins_res_df = pd.DataFrame(list_ins_res)
    return list_ins_res_df

In [ ]:
def mergeDocumentRes(ins_df, val_id_df, threshold=0.0001, debug=False):
    STRIDE = 128
    list_doc_lan = []
    for idx, doc in val_id_df.iterrows():
        doc_id = doc['example_id']
        ins_of_doc = ins_df.loc[ins_df['example_id'] == doc_id]
        
        start_ins = ins_of_doc.loc[ins_of_doc['start'] != 0]
        stop_ins = ins_of_doc.loc[ins_of_doc['stop'] != 0]
        all_non_zero = pd.concat([start_ins,stop_ins]).drop_duplicates()
        
        best_start = -1
        best_stop = -1
        best_target = 0
        best_score = threshold
                    
        for idx_ins, ins in all_non_zero.iterrows():
            ins_start = int(ins['start'])
            ins_stop = int(ins['stop'])
            ins_target = int(ins['target'])
            
            part_id = ins['part_id']
            part_start = part_id*STRIDE
            
            real_start = int(ins_start + part_start)
            real_stop = int(ins_stop + part_start)
            
            s_start = ins['start_score']
            s_stop = ins['stop_score']
            
            cls_start = ins['start_CLS']
            cls_stop = ins['stop_CLS']
            
            if real_stop > real_start:   
                if s_start - cls_start + s_stop - cls_stop > best_score:
                    best_score = s_start - cls_start + s_stop - cls_stop
                    best_start = real_start
                    best_stop = real_stop
                    best_target = ins_target

        doc_lan = {}
        doc_lan['example_id'] = doc_id
        doc_lan['start'] = best_start
        doc_lan['stop'] = best_stop
        doc_lan['target'] = best_target
        doc_lan['score'] = best_score
        
        if debug:
            if idx == 101:
                print(doc_lan)
        
        list_doc_lan.append(doc_lan)
    
    list_doc_lan_df = pd.DataFrame(list_doc_lan)
    return list_doc_lan_df

In [ ]:
val_cand_df.head()

,example_id,new_candidates,old_candidates
0,4158175306918787233,"[{'end_token': 169, 'start_token': 6}, {'end_t...","[{'start_token': 8, 'top_level': True, 'end_to..."
1,-1957133654292017851,"[{'end_token': 138, 'start_token': 24}, {'end_...","[{'start_token': 26, 'top_level': True, 'end_t..."
2,-1273245364552286191,"[{'end_token': 146, 'start_token': 12}, {'end_...","[{'start_token': 14, 'top_level': True, 'end_t..."
3,-8663891343543535834,"[{'end_token': 188, 'start_token': 64}, {'end_...","[{'start_token': 66, 'top_level': True, 'end_t..."
4,8161832609608306276,"[{'end_token': 74, 'start_token': 6}, {'end_to...","[{'start_token': 8, 'top_level': True, 'end_to..."


In [ ]:
AnswerTypeRev = {
    0: 'NO_ANSWER',
    1: 'YES',
    2: 'NO',
    3: 'SHORT',
    4: 'LONG'
}

def getSubmission(doc_res_df, doc_cand_df, threshold=0.0001, debug=False):
    doc_res_df.example_id = doc_res_df.example_id.astype(str)
    doc_cand_df.example_id = doc_cand_df.example_id.astype(str)
    if debug:
        print(doc_res_df.dtypes)
        print(doc_cand_df.dtypes)

    combine_df = pd.merge(doc_res_df, doc_cand_df, on='example_id')
    lines = []
    for id, doc in combine_df.iterrows():

        example_id = doc['example_id']
        long_id = str(example_id) + '_long'
        short_id = str(example_id) + '_short'

        line_long = {}
        line_long['example_id'] = long_id

        an_start = int(doc['start'])
        an_stop = int(doc['stop'])
        an_target = doc['target']
        an_score = doc['score']
        # print(an_start, an_stop, an_target, an_score)
        lan_start, lan_stop = -1, -1

        # find long answer 
        if an_start > 0 and an_stop > 0:
            candidates = doc['new_candidates']
            an_range = [*range(an_start, an_stop + 1, 1)]

            best_inter = 0.5
            shortest = 10000000000000
            best_id = 0
            for cidx, cand in enumerate(candidates):
                c_start = int(cand['start_token'])
                c_stop = int(cand['end_token'])

                c_range = [*range(c_start, c_stop + 1, 1)]
                inter = len(list(set(an_range)&set(c_range)))
            
                if float(inter) > best_inter:
                    best_id = cidx
                    best_inter = inter
                    shortest = len(c_range)
                elif inter == best_inter:
                    if shortest > len(c_range):
                        best_id = cidx
                        shortest = len(c_range)

            real_candidates = doc['old_candidates']
            lan_start = real_candidates[best_id]['start_token']
            lan_stop = real_candidates[best_id]['end_token']

            if debug:
                if id == 101:
                    print(lan_start, lan_stop)

        if lan_start > 0 and lan_stop > 0:
            long_string = str(lan_start) + ':' + str(lan_stop)
        else:
            long_string = ''


        line_long['PredictionString'] = long_string
        lines.append(line_long)

    lines_df = pd.DataFrame(lines)
    sorted_df = lines_df.sort_values('example_id')
    return sorted_df

In [ ]:
def getResult(gt_df, res_df, debug=False):
    gt_df.fillna('', inplace=True)
    if gt_df.shape[0] != res_df.shape[0]:
        print('ERROR: Different number of rows')
        return -1.0
    
    TP, TN, FP, FN = 0, 0, 0, 0

    # list_incorrect = []

    gt_id = gt_df['example_id'].astype(str).tolist()
    res_id = res_df['example_id'].astype(str).tolist()

    if gt_id != res_id:
        print("ERROR: Example_id lists are not the same")
        return -1.0
    
    gt_res = gt_df['PredictionString'].tolist()
    res_res = res_df['PredictionString'].tolist()
    id_list = gt_df['example_id'].tolist()
    for i in range(len(gt_res)):
        if gt_res[i] == res_res[i]:
            if gt_res[i] != "":
                TP += 1
            else:
                TN += 1
        else:
            if res_res[i] == '':
                FN += 1
            else:
                FP += 1
    recall = float(TP)/float(TP+FN + 0.000001)
    precision = float(TP)/float(TP+FP + 0.000001)
    print("TP: ", TP)
    print("TN: ", TN)
    print("FP: ", FP)
    print("FN: ", FN)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1: ", float(2*recall*precision)/float(recall + precision + 0.00000001))


In [ ]:
class IntervalEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.x_val, self.y_val = validation_data
        self.list_threshold = [0.0001, 0.001, 0.01, 0.1, 0.15, 0.2]

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=1)
            
            fineResDf = getFineInsRes(y_pred, val_ins_list)

            for threshold in self.list_threshold:
                print('THRESHOLD: ', threshold)
                docAnsDf = mergeDocumentRes(fineResDf, val_id_df, threshold=threshold)
                sub = getSubmission(docAnsDf, val_cand_df)
                getResult(gt_df, sub)

In [ ]:
x1, x2, x3, y = preprocess_data(tran_ins_list, tokenizer, debug=True)

100%|██████████| 577571/577571 [28:38<00:00, 336.07it/s]


In [ ]:
x_val1, x_val2, x_val3, y_val = preprocess_data(val_ins_list, tokenizer)

100%|██████████| 169721/169721 [07:52<00:00, 358.91it/s]


In [11]:
strategy = get_strategy()
weight_path = '/content/drive/MyDrive/data/models/squad/weights-18-0.644.h5'
# weight_path=''
with strategy.scope():
    myQAModel = create_model(tokenizer, model_name)

    if os.path.isfile(weight_path):
        x = np.ones([1, 512], dtype=int)
        myQAModel.predict([x, x, x])
        myQAModel.load_weights(weight_path)
        myQAModel.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    lossSCE = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metricSCA = tf.keras.metrics.SparseCategoricalAccuracy()
    myQAModel.compile(optimizer=opt, loss=lossSCE, metrics=[metricSCA], run_eagerly=False)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.54.131.34:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.54.131.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.54.131.34:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at /content/drive/MyDrive/data/bert_base_uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /content/drive/MyDrive/data/bert_base_uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further

Model: "my_qa_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model (TFBertModel)  multiple                  109484544 
_________________________________________________________________
dense (Dense)                multiple                  769       
_________________________________________________________________
dense_1 (Dense)              multiple                  769       
_________________________________________________________________
dense_2 (Dense)              multiple                  3845      
Total params: 109,489,927
Trainable params: 109,489,927
Non-trainable params: 0
_________________________________________________________________


In [ ]:
save_locally = tf.train.CheckpointOptions(experimental_io_device='/job:localhost')

filepath="/content/drive/MyDrive/data/models/current/weights-{epoch:02d}.h5"
checkpoint_dir = os.path.dirname(filepath)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                verbose=1, 
                                                save_best_only=False, 
                                                save_weights_only=True,
                                                options=save_locally)
# eval = IntervalEvaluation(validation_data=([x_val1, x_val2, x_val3], y_val))

In [ ]:
history = myQAModel.fit(x=[x1, x2, x3],
                        y=y,
                        batch_size=128,
                        # callbacks=[checkpoint, eval], 
                        callbacks=[checkpoint],
                        epochs=20 
                        # validation_data=([x_val1, x_val2, x_val3], y_val), 
                        # validation_batch_size=128,
                        )

Epoch 1/20


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]


4513/4513 [==============================] - 2199s 468ms/step - loss: 2.0567 - sparse_categorical_accuracy: 0.5450

Epoch 00001: saving model to /content/drive/MyDrive/data/models/current/weights-01.h5
Epoch 2/20
4513/4513 [==============================] - 2060s 456ms/step - loss: 1.5609 - sparse_categorical_accuracy: 0.6050

Epoch 00002: saving model to /content/drive/MyDrive/data/models/current/weights-02.h5
Epoch 3/20
4513/4513 [==============================] - 2061s 457ms/step - loss: 1.3012 - sparse_categorical_accuracy: 0.6490

Epoch 00003: saving model to /content/drive/MyDrive/data/models/current/weights-03.h5
Epoch 4/20
4513/4513 [==============================] - 2062s 457ms/step - loss: 1.1125 - sparse_categorical_accuracy: 0.6852

Epoch 00004: saving model to /content/drive/MyDrive/data/models/current/weights-04.h5
Epoch 5/20
4513/4513 [==============================] - 2063s 457ms/step - loss: 0.9764 - sparse_categorical_accuracy: 0.7136

Epoch 00005: saving model to /co

## **Train squad**

In [ ]:
save_locally = tf.train.CheckpointOptions(experimental_io_device='/job:localhost')


filepath="/content/drive/MyDrive/data/models/current/weights-{epoch:02d}-{val_sparse_categorical_accuracy:.3f}.h5"
checkpoint_dir = os.path.dirname(filepath)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                monitor='val_sparse_categorical_accuracy', 
                                                verbose=1, 
                                                save_best_only=True, 
                                                mode='max',
                                                save_weights_only=True,
                                                options=save_locally)
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10)

In [ ]:
history = myQAModel.fit(x=[x1, x2, x3],
                        y=y,
                        batch_size=128,
                        callbacks=[checkpoint, earlyStop], 
                        epochs= 100, 
                        # validation_data=([x_val1, x_val2, x_val3], y_val), 
                        # validation_batch_size=128)
                        validation_split = 0.1)

Epoch 1/100


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]


917/917 [==============================] - ETA: 0s - loss: 2.1526 - sparse_categorical_accuracy: 0.4894

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 3) dtype=int32>]


917/917 [==============================] - 604s 558ms/step - loss: 2.1526 - sparse_categorical_accuracy: 0.4894 - val_loss: 1.6904 - val_sparse_categorical_accuracy: 0.5349

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.53489, saving model to /content/drive/MyDrive/data/models/current/weights-01-0.535.h5
Epoch 2/100
917/917 [==============================] - 435s 475ms/step - loss: 1.5403 - sparse_categorical_accuracy: 0.5579 - val_loss: 1.4841 - val_sparse_categorical_accuracy: 0.5717

Epoch 00002: val_sparse_categorical_accuracy improved from 0.53489 to 0.57166, saving model to /content/drive/MyDrive/data/models/current/weights-02-0.572.h5
Epoch 3/100
917/917 [==============================] - 435s 475ms/step - loss: 1.2392 - sparse_categorical_accuracy: 0.6139 - val_loss: 1.3919 - val_sparse_categorical_accuracy: 0.5900

Epoch 00003: val_sparse_categorical_accuracy improved from 0.57166 to 0.58996, saving model to /content/drive/MyDrive/data/models/current/wei